In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path

In [2]:
df = pd.read_csv(Path('./data/20201004.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],Rank your movie choices. You don't have to do a full ranking. [Fourth Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice],What theme should we explore next week? [Fifth Choice]
0,9/29/2020 20:24:00,Touch of Evil (1958),The Killing Of A Chinese Bookie (1976),Mr. Smith Goes to Washington (1939),Metropolis (1927),SEX + DEATH,UNSUPERVISED CHILDREN,WEIRD SCIENCE,INDIA!,No Theme/Free-for-all
1,9/29/2020 20:37:22,Metropolis (1927),Mr. Smith Goes to Washington (1939),NaN,NaN,SEX + DEATH,UNSUPERVISED CHILDREN,WEIRD SCIENCE,INDIA!,No Theme/Free-for-all
2,9/29/2020 21:40:55,Mr. Smith Goes to Washington (1939),Metropolis (1927),Touch of Evil (1958),The Killing Of A Chinese Bookie (1976),INDIA!,UNSUPERVISED CHILDREN,SEX + DEATH,WEIRD SCIENCE,NaN
3,9/29/2020 22:26:10,Mr. Smith Goes to Washington (1939),The Killing Of A Chinese Bookie (1976),Metropolis (1927),Touch of Evil (1958),SEX + DEATH,UNSUPERVISED CHILDREN,INDIA!,WEIRD SCIENCE,No Theme/Free-for-all
4,9/30/2020 16:19:15,Touch of Evil (1958),The Killing Of A Chinese Bookie (1976),NaN,NaN,WEIRD SCIENCE,INDIA!,UNSUPERVISED CHILDREN,SEX + DEATH,NaN
5,10/1/2020 1:06:38,Mr. Smith Goes to Washington (1939),Touch of Evil (1958),Metropolis (1927),The Killing Of A Chinese Bookie (1976),UNSUPERVISED CHILDREN,SEX + DEATH,INDIA!,WEIRD SCIENCE,No Theme/Free-for-all
6,10/1/2020 10:51:45,Touch of Evil (1958),Metropolis (1927),Mr. Smith Goes to Washington (1939),The Killing Of A Chinese Bookie (1976),UNSUPERVISED CHILDREN,WEIRD SCIENCE,INDIA!,SEX + DEATH,No Theme/Free-for-all
7,10/1/2020 11:09:07,The Killing Of A Chinese Bookie (1976),Metropolis (1927),Mr. Smith Goes to Washington (1939),Touch of Evil (1958),UNSUPERVISED CHILDREN,SEX + DEATH,WEIRD SCIENCE,No Theme/Free-for-all,INDIA!


In [3]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [4]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['Touch of Evil (1958)', 'The Killing Of A Chinese Bookie (1976)',
       'Mr. Smith Goes to Washington (1939)', 'Metropolis (1927)'],
      dtype=object)

In [5]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [6]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [7]:
submission_dict

{'Touch of Evil (1958)': <Candidate('Touch of Evil (1958)')>,
 'The Killing Of A Chinese Bookie (1976)': <Candidate('The Killing Of A Chinese Bookie (1976)')>,
 'Mr. Smith Goes to Washington (1939)': <Candidate('Mr. Smith Goes to Washington (1939)')>,
 'Metropolis (1927)': <Candidate('Metropolis (1927)')>}

In [8]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [9]:
ballots

[<Ballot(Touch of Evil (1958), The Killing Of A Chinese Bookie (1976), Mr. Smith Goes to Washington (1939), Metropolis (1927))>,
 <Ballot(Metropolis (1927), Mr. Smith Goes to Washington (1939))>,
 <Ballot(Mr. Smith Goes to Washington (1939), Metropolis (1927), Touch of Evil (1958), The Killing Of A Chinese Bookie (1976))>,
 <Ballot(Mr. Smith Goes to Washington (1939), The Killing Of A Chinese Bookie (1976), Metropolis (1927), Touch of Evil (1958))>,
 <Ballot(Touch of Evil (1958), The Killing Of A Chinese Bookie (1976))>,
 <Ballot(Mr. Smith Goes to Washington (1939), Touch of Evil (1958), Metropolis (1927), The Killing Of A Chinese Bookie (1976))>,
 <Ballot(Touch of Evil (1958), Metropolis (1927), Mr. Smith Goes to Washington (1939), The Killing Of A Chinese Bookie (1976))>,
 <Ballot(The Killing Of A Chinese Bookie (1976), Metropolis (1927), Mr. Smith Goes to Washington (1939), Touch of Evil (1958))>]

In [10]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [11]:
print(election_result)

ROUND 1
Candidate                                 Votes  Status
--------------------------------------  -------  --------
Mr. Smith Goes to Washington (1939)           3  Hopeful
Touch of Evil (1958)                          3  Hopeful
Metropolis (1927)                             1  Rejected
The Killing Of A Chinese Bookie (1976)        1  Rejected

FINAL RESULT
Candidate                                 Votes  Status
--------------------------------------  -------  --------
Mr. Smith Goes to Washington (1939)           5  Elected
Touch of Evil (1958)                          3  Rejected
Metropolis (1927)                             0  Rejected
The Killing Of A Chinese Bookie (1976)        0  Rejected



In [12]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

ROUND 1
Candidate                Votes  Status
---------------------  -------  --------
UNSUPERVISED CHILDREN        3  Hopeful
SEX + DEATH                  3  Hopeful
WEIRD SCIENCE                1  Rejected
INDIA!                       1  Rejected
No Theme/Free-for-all        0  Rejected

FINAL RESULT
Candidate                Votes  Status
---------------------  -------  --------
UNSUPERVISED CHILDREN        5  Elected
SEX + DEATH                  3  Rejected
WEIRD SCIENCE                0  Rejected
INDIA!                       0  Rejected
No Theme/Free-for-all        0  Rejected

